In [79]:
import boto3
import pandas as pd
from io import StringIO
import time

In [ ]:
import boto3
import json
import os

client = boto3.client('secretsmanager')


response = client.get_secret_value(SecretId='my-aws-credentials')
secret = json.loads(response['SecretString'])


os.environ['AWS_ACCESS_KEY_ID'] = secret['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = secret['AWS_SECRET_ACCESS_KEY']



In [81]:
import boto3

session = boto3.Session(region_name = 'ap-south-1',aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'))
print('successfully aws session created')

successfully aws session created


In [82]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_bucket(bucket_name, region="ap-south-1"):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [6]:
create_bucket("sundardl-travel-data")

True

In [83]:
import boto3
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  sundar-test-bucket11
  sundardl-travel-data


In [ ]:
import boto3

s3 = boto3.client('s3')

bucket_name = 'sundardl-travel-data'
folder_name = 'flights/'

response = s3.put_object(Bucket=bucket_name, Key=folder_name)

print(response)

{'ResponseMetadata': {'RequestId': 'FAS03VWP6QY4VN1Y', 'HostId': 'Gw9bhkCwCe3l6wgOd7Y3rgavyIon6xcWx4p3tLqpph3UCH4hON/JWGlmzM2vcbDkg0XPcRTCBRJyX5I59ylwEQ==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Gw9bhkCwCe3l6wgOd7Y3rgavyIon6xcWx4p3tLqpph3UCH4hON/JWGlmzM2vcbDkg0XPcRTCBRJyX5I59ylwEQ==', 'x-amz-request-id': 'FAS03VWP6QY4VN1Y', 'date': 'Sat, 28 Jun 2025 10:06:10 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"d41d8cd98f00b204e9800998ecf8427e"', 'x-amz-checksum-crc32': 'AAAAAA==', 'x-amz-checksum-type': 'FULL_OBJECT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumCRC32': 'AAAAAA==', 'ChecksumType': 'FULL_OBJECT', 'ServerSideEncryption': 'AES256'}


In [ ]:
import boto3


s3 = boto3.client('s3')


bucket_name = 'sundardl-travel-data'
folder_name = 'flights'


file_path = r"D:\study materials\aws\archive\flights.csv"

s3_file_name = f'{folder_name}/users.csv'
s3.upload_file(file_path, bucket_name, s3_file_name)

ATHENA

In [ ]:
AWS_REGION = "ap-south-1"
SCHEMA_NAME = "travel_dataset"
S3_STAGING_DIR = "s3://sundar-test-bucket11/output/"
S3_BUCKET_NAME = "sundar-test-bucket11"
S3_OUTPUT_DIRECTORY = "output"

In [ ]:
# creates a Python client object to interact with Amazon Athena using the boto3 library
athena_client = boto3.client(
    "athena",
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
    region_name = AWS_REGION,
)

In [ ]:
'''Run the SQL query SELECT * FROM users in the database SCHEMA_NAME,
and store the results in the S3 folder S3_STAGING_DIR, using server-side encryption.''' 

response = athena_client.start_query_execution(
    QueryString = "Select * from users",
    QueryExecutionContext = {"Database":SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration" : {"EncryptionOption" : "SSE_S3"}
    },
)

In [7]:
response

{'QueryExecutionId': '8e428e9d-53be-43a4-80a6-c36ee4c4f8ee',
 'ResponseMetadata': {'RequestId': 'f66eda11-0a95-4323-aee8-ecc851886f56',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 28 Jun 2025 11:20:15 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f66eda11-0a95-4323-aee8-ecc851886f56'},
  'RetryAttempts': 0}}


This function waits for an Athena query to complete, downloads the result CSV file from S3,
and loads it into a pandas DataFrame.

Here's how it works:
1. It uses the QueryExecutionId from the Athena response to keep checking until the query is done.
2. Once completed, it connects to S3 using boto3 and downloads the result file stored by Athena.
   Athena automatically stores query results in the S3 path given during query submission.
3. It downloads the result file (a CSV) using the same QueryExecutionId as the filename.
4. Finally, it reads the downloaded CSV into a pandas DataFrame and returns it for further use.


In [ ]:
Dict = {} 

def download_and_load_query_results(client: boto3.client, query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err

    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)


In [8]:
users = download_and_load_query_results(athena_client, response)

In [9]:
users.head(10)

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44
5,5,4You,Trina Thomas,none,47
6,6,4You,Jesse Decelle,male,46
7,7,4You,Gregoria Gil,female,21
8,8,4You,Jack Sabo,none,41
9,9,4You,Debbie Helms,none,35


In [10]:
users.shape

(1340, 5)

In [11]:
response = athena_client.start_query_execution(
    QueryString = "Select * from flights",
    QueryExecutionContext = {"Database":SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration" : {"EncryptionOption" : "SSE_S3"}
    },
)

In [ ]:
response

In [ ]:
flights = download_and_load_query_results(athena_client, response)

In [ ]:
flights.head(10)

In [ ]:
flights.shape

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "Select * from hotels",
    QueryExecutionContext = {"Database":SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration" : {"EncryptionOption" : "SSE_S3"}
    },
)

In [ ]:
response

In [ ]:
hotels = download_and_load_query_results(athena_client, response)

In [ ]:
hotels.tail()

In [ ]:
hotels.shape

In [64]:
import pandas as pd
flights  = pd.read_csv(r"D:\study materials\aws\archive\flights.csv")
flights.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [65]:
import pandas as pd
hotels  = pd.read_csv(r"D:\study materials\aws\archive\hotels.csv")
hotels.head()

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,09/26/2019
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,10/10/2019
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,11/14/2019
3,11,0,Hotel K,Salvador (BH),4,263.41,1053.64,12/12/2019
4,13,0,Hotel A,Florianopolis (SC),1,313.02,313.02,12/26/2019


In [66]:
import pandas as pd
users  = pd.read_csv(r"D:\study materials\aws\archive\users.csv")
users.head()

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


# **user**

In [67]:
users.head()

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


In [68]:
user = users
user = user.rename(columns={'code': 'userCode'})
user.head()

,userCode,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


In [69]:
#user = user.rename(columns={'code': 'userCode'})
#user  = user[['userCode', 'name','age']]
#user.head(1)

In [70]:
user.shape

(1340, 5)

In [71]:
type(user)

pandas.core.frame.DataFrame

In [133]:
user

,userCode,name,age,gender,companyCode
0,0,Roy Braun,21,M,700
1,1,Joseph Holsten,37,M,700
2,2,Wilma Mcinnis,48,F,700
3,3,Paula Daniel,23,F,700
4,4,Patricia Carson,44,F,700
...,...,...,...,...,...
1335,1335,Albert Garroutte,23,M,704
1336,1336,Kim Shores,40,F,704
1337,1337,James Gimenez,28,M,704
1338,1338,Viola Agosta,52,F,704


In [72]:
import pandas as pd

company = users
company = company.rename(columns={'company': 'companyName'})

# Sample company names
company = pd.DataFrame({
    'companyName': ['4You', 'Monsters CYA', 'Wonka Company', 'Acme Factory', 'Umbrella LTDA']
})

# Generate dynamic company codes starting from 700
company['companyCode'] = range(700, 700 + len(company))

# Reorder columns
company = company[['companyCode', 'companyName']]

print(company)


   companyCode    companyName
0          700           4You
1          701   Monsters CYA
2          702  Wonka Company
3          703   Acme Factory
4          704  Umbrella LTDA


In [73]:
user= pd.merge(user, company, left_on='company', right_on='companyName', how='left')
user = user[['userCode', 'name', 'age', 'gender','companyCode']]


In [74]:
user['gender'] = user['gender'].replace({'male': 'M', 'female': 'F'})
user['gender'] = user['gender'].fillna('unknown')                 # handles NaN
user['gender'] = user['gender'].replace('', 'unknown')           # handles empty strings


In [75]:
user

,userCode,name,age,gender,companyCode
0,0,Roy Braun,21,M,700
1,1,Joseph Holsten,37,M,700
2,2,Wilma Mcinnis,48,F,700
3,3,Paula Daniel,23,F,700
4,4,Patricia Carson,44,F,700
...,...,...,...,...,...
1335,1335,Albert Garroutte,23,M,704
1336,1336,Kim Shores,40,F,704
1337,1337,James Gimenez,28,M,704
1338,1338,Viola Agosta,52,F,704


# **gender**

In [76]:
gender = users
gender = gender.rename(columns={'gender': 'genderName'})
gender.head(10)

,code,company,name,genderName,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44
5,5,4You,Trina Thomas,none,47
6,6,4You,Jesse Decelle,male,46
7,7,4You,Gregoria Gil,female,21
8,8,4You,Jack Sabo,none,41
9,9,4You,Debbie Helms,none,35


In [77]:
import pandas as pd

# Assuming you have a DataFrame 'df' with the columns 'code', 'company', 'genderName', 'gender', 'age'
# Read your CSV data into 'df'

# Define a function to map gender to genderCode
def map_gender_to_code(gender):
    if gender == 'male':
        return 'M'
    elif gender == 'female':
        return 'F'
    else:
        return 'Unknown'  # You might want to handle other cases as well

# Apply the function to create a new column 'genderCode'
gender['genderCode'] = gender['genderName'].apply(lambda x: map_gender_to_code(x))

# Print the updated DataFrame
print(gender.head(10))

   code company             name genderName  age genderCode
0     0    4You        Roy Braun       male   21          M
1     1    4You   Joseph Holsten       male   37          M
2     2    4You    Wilma Mcinnis     female   48          F
3     3    4You     Paula Daniel     female   23          F
4     4    4You  Patricia Carson     female   44          F
5     5    4You     Trina Thomas       none   47    Unknown
6     6    4You    Jesse Decelle       male   46          M
7     7    4You     Gregoria Gil     female   21          F
8     8    4You        Jack Sabo       none   41    Unknown
9     9    4You     Debbie Helms       none   35    Unknown


In [78]:
gender = gender[['genderCode', 'genderName']]
gender.head(10)

,genderCode,genderName
0,M,male
1,M,male
2,F,female
3,F,female
4,F,female
5,Unknown,none
6,M,male
7,F,female
8,Unknown,none
9,Unknown,none


In [79]:
gender = gender[['genderCode', 'genderName']].drop_duplicates().reset_index(drop = True)

In [80]:
gender.shape

(3, 2)

# **location**

In [81]:
location = flights
location = location.rename(columns={'from': 'locationName'})
location.shape

(271888, 10)

In [82]:
location.tail()

,travelCode,userCode,locationName,to,flightType,price,time,distance,agency,date
271883,135941,1339,Campo Grande (MS),Florianopolis (SC),firstClass,1446.34,1.49,573.81,CloudFy,07/12/2020
271884,135942,1339,Florianopolis (SC),Natal (RN),economic,726.95,1.84,709.37,CloudFy,07/16/2020
271885,135942,1339,Natal (RN),Florianopolis (SC),economic,873.07,1.84,709.37,CloudFy,07/20/2020
271886,135943,1339,Florianopolis (SC),Rio de Janeiro (RJ),economic,313.62,1.21,466.30,CloudFy,07/23/2020
271887,135943,1339,Rio de Janeiro (RJ),Florianopolis (SC),economic,533.69,1.21,466.30,CloudFy,07/26/2020


In [83]:
flights.shape

(271888, 10)

In [84]:
import pandas as pd

# Assuming you have a DataFrame 'company' with a column named 'companyname'
location = location['locationName'].drop_duplicates().reset_index(drop=True)
location

0            Recife (PE)
1     Florianopolis (SC)
2          Brasilia (DF)
3           Aracaju (SE)
4          Salvador (BH)
5      Campo Grande (MS)
6         Sao Paulo (SP)
7             Natal (RN)
8    Rio de Janeiro (RJ)
Name: locationName, dtype: object

In [85]:
location = pd.DataFrame({'locationCode': range(1, len(location)+1), 'locationName': location})

location['locationName'] = location['locationName'].str.upper()

location

,locationCode,locationName
0,1,RECIFE (PE)
1,2,FLORIANOPOLIS (SC)
2,3,BRASILIA (DF)
3,4,ARACAJU (SE)
4,5,SALVADOR (BH)
5,6,CAMPO GRANDE (MS)
6,7,SAO PAULO (SP)
7,8,NATAL (RN)
8,9,RIO DE JANEIRO (RJ)


# **Accommodation**

In [86]:
hotels.head()

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,09/26/2019
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,10/10/2019
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,11/14/2019
3,11,0,Hotel K,Salvador (BH),4,263.41,1053.64,12/12/2019
4,13,0,Hotel A,Florianopolis (SC),1,313.02,313.02,12/26/2019


In [87]:
accommodation = hotels
accommodation.head(10)

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,09/26/2019
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,10/10/2019
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,11/14/2019
3,11,0,Hotel K,Salvador (BH),4,263.41,1053.64,12/12/2019
4,13,0,Hotel A,Florianopolis (SC),1,313.02,313.02,12/26/2019
5,15,0,Hotel BD,Natal (RN),2,242.88,485.76,01/09/2020
6,22,0,Hotel Z,Aracaju (SE),2,208.04,416.08,02/27/2020
7,29,0,Hotel AU,Recife (PE),4,312.83,1251.32,04/16/2020
8,32,0,Hotel AF,Sao Paulo (SP),2,139.10,278.20,05/07/2020
9,33,0,Hotel K,Salvador (BH),4,263.41,1053.64,05/14/2020


In [88]:
accommodation.shape

(40552, 8)

In [89]:
type(accommodation)

pandas.core.frame.DataFrame

In [90]:
#accommodation['accommodationCode'] = accommodation.groupby('name').ngroup() + 1
#accommodation = accommodation[['accommodationCode', 'travelcode', 'usercode', 'name', 'place', 'days', 'price', 'total', 'date']]
#accommodation.head(3)

In [91]:
type(accommodation)

pandas.core.frame.DataFrame

In [92]:
accommodation['date'] = pd.to_datetime(accommodation['date'])
accommodation.head(3)

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,2019-09-26
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,2019-10-10
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,2019-11-14


In [93]:
import pandas as pd

# Hotel names
hotel = pd.DataFrame({
    'hotelName': ['Hotel A', 'Hotel K', 'Hotel BD', 'Hotel Z', 'Hotel AU',
                  'Hotel AF', 'Hotel BW', 'Hotel CB', 'Hotel BP']
})

# Generate hotel codes starting from 900
hotel['hotelCode'] = range(900, 900 + len(hotel))

# Reorder columns
hotel = hotel[['hotelCode', 'hotelName']]

print(hotel)


   hotelCode hotelName
0        900   Hotel A
1        901   Hotel K
2        902  Hotel BD
3        903   Hotel Z
4        904  Hotel AU
5        905  Hotel AF
6        906  Hotel BW
7        907  Hotel CB
8        908  Hotel BP


In [94]:
accommodation = pd.merge(accommodation, hotel, left_on='name', right_on='hotelName', how='left')
accommodation.tail()

,travelCode,userCode,name,place,days,price,total,date,hotelCode,hotelName
40547,135938,1339,Hotel BP,Brasilia (DF),3,247.62,742.86,2020-06-18,908,Hotel BP
40548,135939,1339,Hotel BP,Brasilia (DF),1,247.62,247.62,2020-06-25,908,Hotel BP
40549,135940,1339,Hotel BW,Campo Grande (MS),3,60.39,181.17,2020-07-02,906,Hotel BW
40550,135941,1339,Hotel BW,Campo Grande (MS),3,60.39,181.17,2020-07-09,906,Hotel BW
40551,135942,1339,Hotel BD,Natal (RN),4,242.88,971.52,2020-07-16,902,Hotel BD


In [95]:
type(accommodation)

pandas.core.frame.DataFrame

In [96]:
import pandas as pd

# Assuming you have DataFrames 'accommodation' and 'locations'
# 'accommodation' contains columns: 'accommodationCode', 'travelCode', 'userCode', 'name', 'place', 'days', 'price', 'total', 'date'
# 'locations' contains columns: 'locationCode', 'locationName'

# Convert 'locationName' and 'place' columns to lowercase for case-insensitive matching
location['locationName'] = location['locationName'].str.upper()
accommodation['place'] = accommodation['place'].str.upper()

# Merge DataFrames based on 'locationName' and 'place'
result_df = pd.merge(accommodation, location, left_on='place', right_on='locationName', how='outer')

# Drop the duplicate columns (locationName and locationCode)
result_df = result_df.drop([ 'locationName'], axis=1)

# Print the result DataFrame
result_df.head(10)

,travelCode,userCode,name,place,days,price,total,date,hotelCode,hotelName,locationCode
0,22,0,Hotel Z,ARACAJU (SE),2,208.04,416.08,2020-02-27,903,Hotel Z,4
1,57,0,Hotel Z,ARACAJU (SE),4,208.04,832.16,2020-10-29,903,Hotel Z,4
2,60,0,Hotel Z,ARACAJU (SE),4,208.04,832.16,2020-11-19,903,Hotel Z,4
3,105,2,Hotel Z,ARACAJU (SE),4,208.04,832.16,2019-12-05,903,Hotel Z,4
4,124,2,Hotel Z,ARACAJU (SE),3,208.04,624.12,2020-04-16,903,Hotel Z,4
5,150,2,Hotel Z,ARACAJU (SE),2,208.04,416.08,2020-10-15,903,Hotel Z,4
6,177,2,Hotel Z,ARACAJU (SE),2,208.04,416.08,2021-04-22,903,Hotel Z,4
7,187,2,Hotel Z,ARACAJU (SE),3,208.04,624.12,2021-07-01,903,Hotel Z,4
8,211,2,Hotel Z,ARACAJU (SE),3,208.04,624.12,2021-12-16,903,Hotel Z,4
9,338,3,Hotel Z,ARACAJU (SE),3,208.04,624.12,2021-11-18,903,Hotel Z,4


In [97]:
accommodation = result_df[["userCode",  "hotelCode", "locationCode", "name", "days", "price", "total", "date"]]
accommodation.loc[3500:3510]

,userCode,hotelCode,locationCode,name,days,price,total,date
3500,1161,903,4,Hotel Z,4,208.04,832.16,2020-02-20
3501,1161,903,4,Hotel Z,1,208.04,208.04,2020-07-16
3502,1161,903,4,Hotel Z,3,208.04,624.12,2020-08-27
3503,1162,903,4,Hotel Z,2,208.04,416.08,2020-05-14
3504,1162,903,4,Hotel Z,2,208.04,416.08,2020-06-25
3505,1162,903,4,Hotel Z,4,208.04,832.16,2020-09-24
3506,1162,903,4,Hotel Z,1,208.04,208.04,2020-11-19
3507,1162,903,4,Hotel Z,4,208.04,832.16,2021-04-08
3508,1162,903,4,Hotel Z,3,208.04,624.12,2021-11-25
3509,1162,903,4,Hotel Z,3,208.04,624.12,2022-02-10


In [98]:
accommodation.insert(0, "accommodationId", range(1, len(accommodation)+1))


In [99]:
accommodation.loc[0:5]

,accommodationId,userCode,hotelCode,locationCode,name,days,price,total,date
0,1,0,903,4,Hotel Z,2,208.04,416.08,2020-02-27
1,2,0,903,4,Hotel Z,4,208.04,832.16,2020-10-29
2,3,0,903,4,Hotel Z,4,208.04,832.16,2020-11-19
3,4,2,903,4,Hotel Z,4,208.04,832.16,2019-12-05
4,5,2,903,4,Hotel Z,3,208.04,624.12,2020-04-16
5,6,2,903,4,Hotel Z,2,208.04,416.08,2020-10-15


# **FlightType**

In [100]:
flighttype = flights

flighttype.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [101]:
flighttype = flighttype.rename(columns={'flightType': 'flightTypeName'})
flighttype.head()

,travelCode,userCode,from,to,flightTypeName,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [102]:
flighttype = flighttype['flightTypeName'].unique()
flighttype

array(['firstClass', 'economic', 'premium'], dtype=object)

In [103]:
import pandas as pd

# Assuming flighttype contains unique flight type names
flighttype = pd.DataFrame({'flightTypeName': ['firstClass', 'economic', 'premium']})

# Create a mapping dictionary
flight_type_mapping = {
    'firstClass': '100',
    'economic': '101',
    'premium': '102'
}

# Add a new column based on the mapping
flighttype['flightTypeCode'] = flighttype['flightTypeName'].map(flight_type_mapping)

# Print the resulting DataFrame
flighttype = flighttype[['flightTypeCode', 'flightTypeName']]


In [104]:
flighttype

,flightTypeCode,flightTypeName
0,100,firstClass
1,101,economic
2,102,premium


# **agency**

In [105]:
agency = flights
agency = agency.rename(columns={'agency': 'agencyName'})
agency.tail()

,travelCode,userCode,from,to,flightType,price,time,distance,agencyName,date
271883,135941,1339,Campo Grande (MS),Florianopolis (SC),firstClass,1446.34,1.49,573.81,CloudFy,07/12/2020
271884,135942,1339,Florianopolis (SC),Natal (RN),economic,726.95,1.84,709.37,CloudFy,07/16/2020
271885,135942,1339,Natal (RN),Florianopolis (SC),economic,873.07,1.84,709.37,CloudFy,07/20/2020
271886,135943,1339,Florianopolis (SC),Rio de Janeiro (RJ),economic,313.62,1.21,466.30,CloudFy,07/23/2020
271887,135943,1339,Rio de Janeiro (RJ),Florianopolis (SC),economic,533.69,1.21,466.30,CloudFy,07/26/2020


In [106]:
agency = agency['agencyName'].unique()
agency

array(['FlyingDrops', 'CloudFy', 'Rainbow'], dtype=object)

In [107]:
import pandas as pd

# Assuming flighttype contains unique flight type names
agency = pd.DataFrame({'agencyName': ['FlyingDrops', 'CloudFy', 'Rainbow']})

# Create a mapping dictionary
agency_mapping = {
    'FlyingDrops': '400',
    'CloudFy': '401',
    'Rainbow': '402'
}

# Add a new column based on the mapping
agency['agencyCode'] = agency['agencyName'].map(agency_mapping)

# Print the resulting DataFrame
agency= agency[['agencyCode', 'agencyName']]
agency

,agencyCode,agencyName
0,400,FlyingDrops
1,401,CloudFy
2,402,Rainbow


# **travel**

In [108]:
travel = flights
travel['from'] = travel['from'].str.upper()

travel['to'] = travel['to'].str.upper()
travel.tail()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
271883,135941,1339,CAMPO GRANDE (MS),FLORIANOPOLIS (SC),firstClass,1446.34,1.49,573.81,CloudFy,07/12/2020
271884,135942,1339,FLORIANOPOLIS (SC),NATAL (RN),economic,726.95,1.84,709.37,CloudFy,07/16/2020
271885,135942,1339,NATAL (RN),FLORIANOPOLIS (SC),economic,873.07,1.84,709.37,CloudFy,07/20/2020
271886,135943,1339,FLORIANOPOLIS (SC),RIO DE JANEIRO (RJ),economic,313.62,1.21,466.30,CloudFy,07/23/2020
271887,135943,1339,RIO DE JANEIRO (RJ),FLORIANOPOLIS (SC),economic,533.69,1.21,466.30,CloudFy,07/26/2020


In [109]:
travel['date'] = pd.to_datetime(travel['date'])
travel.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,RECIFE (PE),FLORIANOPOLIS (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26
1,0,0,FLORIANOPOLIS (SC),RECIFE (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,2019-09-30
2,1,0,BRASILIA (DF),FLORIANOPOLIS (SC),firstClass,1487.52,1.66,637.56,CloudFy,2019-10-03
3,1,0,FLORIANOPOLIS (SC),BRASILIA (DF),firstClass,1127.36,1.66,637.56,CloudFy,2019-10-04
4,2,0,ARACAJU (SE),SALVADOR (BH),firstClass,1684.05,2.16,830.86,CloudFy,2019-10-10


In [110]:
fromLC = pd.merge(  travel, location, left_on='from', right_on='locationName', how='left')

# Drop the duplicate columns (locationName and locationCode)
# result_df = result_df.drop([ 'locationName'], axis=1)

# Print the result DataFrame
fromLC.shape

(271888, 12)

In [111]:
fromLC['date'] = pd.to_datetime(fromLC['date'])
fromLC.loc[1:7]

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,locationCode,locationName
1,0,0,FLORIANOPOLIS (SC),RECIFE (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,2019-09-30,2,FLORIANOPOLIS (SC)
2,1,0,BRASILIA (DF),FLORIANOPOLIS (SC),firstClass,1487.52,1.66,637.56,CloudFy,2019-10-03,3,BRASILIA (DF)
3,1,0,FLORIANOPOLIS (SC),BRASILIA (DF),firstClass,1127.36,1.66,637.56,CloudFy,2019-10-04,2,FLORIANOPOLIS (SC)
4,2,0,ARACAJU (SE),SALVADOR (BH),firstClass,1684.05,2.16,830.86,CloudFy,2019-10-10,4,ARACAJU (SE)
5,2,0,SALVADOR (BH),ARACAJU (SE),firstClass,1531.92,2.16,830.86,CloudFy,2019-10-12,5,SALVADOR (BH)
6,3,0,ARACAJU (SE),CAMPO GRANDE (MS),economic,743.54,1.69,650.10,Rainbow,2019-10-17,4,ARACAJU (SE)
7,3,0,CAMPO GRANDE (MS),ARACAJU (SE),economic,877.56,1.69,650.10,Rainbow,2019-10-20,6,CAMPO GRANDE (MS)


In [112]:
#result_df1[result_df1['locationCode'] == 6]
fromLC = fromLC.rename(columns={'locationCode': 'fromLocationCode'})
fromLC.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,fromLocationCode,locationName
0,0,0,RECIFE (PE),FLORIANOPOLIS (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26,1,RECIFE (PE)
1,0,0,FLORIANOPOLIS (SC),RECIFE (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,2019-09-30,2,FLORIANOPOLIS (SC)
2,1,0,BRASILIA (DF),FLORIANOPOLIS (SC),firstClass,1487.52,1.66,637.56,CloudFy,2019-10-03,3,BRASILIA (DF)
3,1,0,FLORIANOPOLIS (SC),BRASILIA (DF),firstClass,1127.36,1.66,637.56,CloudFy,2019-10-04,2,FLORIANOPOLIS (SC)
4,2,0,ARACAJU (SE),SALVADOR (BH),firstClass,1684.05,2.16,830.86,CloudFy,2019-10-10,4,ARACAJU (SE)


In [113]:
toLC = pd.merge(fromLC, location, left_on='to', right_on='locationName', how='left')
toLC.head(3)

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,fromLocationCode,locationName_x,locationCode,locationName_y
0,0,0,RECIFE (PE),FLORIANOPOLIS (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26,1,RECIFE (PE),2,FLORIANOPOLIS (SC)
1,0,0,FLORIANOPOLIS (SC),RECIFE (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,2019-09-30,2,FLORIANOPOLIS (SC),1,RECIFE (PE)
2,1,0,BRASILIA (DF),FLORIANOPOLIS (SC),firstClass,1487.52,1.66,637.56,CloudFy,2019-10-03,3,BRASILIA (DF),2,FLORIANOPOLIS (SC)


In [114]:
toLC = toLC.rename(columns={'locationCode': 'toLocationCode'})
toLC.tail(3)

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,fromLocationCode,locationName_x,toLocationCode,locationName_y
271885,135942,1339,NATAL (RN),FLORIANOPOLIS (SC),economic,873.07,1.84,709.37,CloudFy,2020-07-20,8,NATAL (RN),2,FLORIANOPOLIS (SC)
271886,135943,1339,FLORIANOPOLIS (SC),RIO DE JANEIRO (RJ),economic,313.62,1.21,466.30,CloudFy,2020-07-23,2,FLORIANOPOLIS (SC),9,RIO DE JANEIRO (RJ)
271887,135943,1339,RIO DE JANEIRO (RJ),FLORIANOPOLIS (SC),economic,533.69,1.21,466.30,CloudFy,2020-07-26,9,RIO DE JANEIRO (RJ),2,FLORIANOPOLIS (SC)


In [115]:
flighttypecode  =  pd.merge(toLC, flighttype, left_on='flightType', right_on='flightTypeName', how='left')
flighttypecode.tail()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,fromLocationCode,locationName_x,toLocationCode,locationName_y,flightTypeCode,flightTypeName
271883,135941,1339,CAMPO GRANDE (MS),FLORIANOPOLIS (SC),firstClass,1446.34,1.49,573.81,CloudFy,2020-07-12,6,CAMPO GRANDE (MS),2,FLORIANOPOLIS (SC),100,firstClass
271884,135942,1339,FLORIANOPOLIS (SC),NATAL (RN),economic,726.95,1.84,709.37,CloudFy,2020-07-16,2,FLORIANOPOLIS (SC),8,NATAL (RN),101,economic
271885,135942,1339,NATAL (RN),FLORIANOPOLIS (SC),economic,873.07,1.84,709.37,CloudFy,2020-07-20,8,NATAL (RN),2,FLORIANOPOLIS (SC),101,economic
271886,135943,1339,FLORIANOPOLIS (SC),RIO DE JANEIRO (RJ),economic,313.62,1.21,466.30,CloudFy,2020-07-23,2,FLORIANOPOLIS (SC),9,RIO DE JANEIRO (RJ),101,economic
271887,135943,1339,RIO DE JANEIRO (RJ),FLORIANOPOLIS (SC),economic,533.69,1.21,466.30,CloudFy,2020-07-26,9,RIO DE JANEIRO (RJ),2,FLORIANOPOLIS (SC),101,economic


In [116]:
agentCode = pd.merge(flighttypecode, agency, left_on='agency', right_on='agencyName', how='left')
agentCode.tail()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date,fromLocationCode,locationName_x,toLocationCode,locationName_y,flightTypeCode,flightTypeName,agencyCode,agencyName
271883,135941,1339,CAMPO GRANDE (MS),FLORIANOPOLIS (SC),firstClass,1446.34,1.49,573.81,CloudFy,2020-07-12,6,CAMPO GRANDE (MS),2,FLORIANOPOLIS (SC),100,firstClass,401,CloudFy
271884,135942,1339,FLORIANOPOLIS (SC),NATAL (RN),economic,726.95,1.84,709.37,CloudFy,2020-07-16,2,FLORIANOPOLIS (SC),8,NATAL (RN),101,economic,401,CloudFy
271885,135942,1339,NATAL (RN),FLORIANOPOLIS (SC),economic,873.07,1.84,709.37,CloudFy,2020-07-20,8,NATAL (RN),2,FLORIANOPOLIS (SC),101,economic,401,CloudFy
271886,135943,1339,FLORIANOPOLIS (SC),RIO DE JANEIRO (RJ),economic,313.62,1.21,466.30,CloudFy,2020-07-23,2,FLORIANOPOLIS (SC),9,RIO DE JANEIRO (RJ),101,economic,401,CloudFy
271887,135943,1339,RIO DE JANEIRO (RJ),FLORIANOPOLIS (SC),economic,533.69,1.21,466.30,CloudFy,2020-07-26,9,RIO DE JANEIRO (RJ),2,FLORIANOPOLIS (SC),101,economic,401,CloudFy


In [117]:
travel = agentCode[["travelCode","userCode","fromLocationCode","toLocationCode","flightTypeCode","price","time","distance","agencyCode","date"]]
travel.head()

,travelCode,userCode,fromLocationCode,toLocationCode,flightTypeCode,price,time,distance,agencyCode,date
0,0,0,1,2,100,1434.38,1.76,676.53,400,2019-09-26
1,0,0,2,1,100,1292.29,1.76,676.53,400,2019-09-30
2,1,0,3,2,100,1487.52,1.66,637.56,401,2019-10-03
3,1,0,2,3,100,1127.36,1.66,637.56,401,2019-10-04
4,2,0,4,5,100,1684.05,2.16,830.86,401,2019-10-10


In [118]:
travel['date'] = pd.to_datetime(travel['date'])
travel.head()

C:\Users\Sundar\AppData\Local\Temp\ipykernel_14284\2719411240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel['date'] = pd.to_datetime(travel['date'])


,travelCode,userCode,fromLocationCode,toLocationCode,flightTypeCode,price,time,distance,agencyCode,date
0,0,0,1,2,100,1434.38,1.76,676.53,400,2019-09-26
1,0,0,2,1,100,1292.29,1.76,676.53,400,2019-09-30
2,1,0,3,2,100,1487.52,1.66,637.56,401,2019-10-03
3,1,0,2,3,100,1127.36,1.66,637.56,401,2019-10-04
4,2,0,4,5,100,1684.05,2.16,830.86,401,2019-10-10


# all

In [154]:
user.head(15)

,userCode,name,age,gender,companyCode
0,0,Roy Braun,21,M,700
1,1,Joseph Holsten,37,M,700
2,2,Wilma Mcinnis,48,F,700
3,3,Paula Daniel,23,F,700
4,4,Patricia Carson,44,F,700
5,5,Trina Thomas,47,none,700
6,6,Jesse Decelle,46,M,700
7,7,Gregoria Gil,21,F,700
8,8,Jack Sabo,41,none,700
9,9,Debbie Helms,35,none,700


In [155]:
company

,companyCode,companyName
0,700,4You
1,701,Monsters CYA
2,702,Wonka Company
3,703,Acme Factory
4,704,Umbrella LTDA


In [156]:
gender.head(10)

,genderCode,genderName
0,M,Roy Braun
1,M,Joseph Holsten
2,F,Wilma Mcinnis
3,F,Paula Daniel
4,F,Patricia Carson
5,Unknown,Trina Thomas
6,M,Jesse Decelle
7,F,Gregoria Gil
8,Unknown,Jack Sabo
9,Unknown,Debbie Helms


In [157]:
location

,locationCode,locationName
0,1,RECIFE (PE)
1,2,FLORIANOPOLIS (SC)
2,3,BRASILIA (DF)
3,4,ARACAJU (SE)
4,5,SALVADOR (BH)
5,6,CAMPO GRANDE (MS)
6,7,SAO PAULO (SP)
7,8,NATAL (RN)
8,9,RIO DE JANEIRO (RJ)


In [158]:
accommodation.loc[13000:13010]

,hotelCode,locationCode,name,days,price,total,date
13000,900,2,Hotel A,4,313.02,1252.08,2023-03-09
13001,900,2,Hotel A,4,313.02,1252.08,2020-09-24
13002,900,2,Hotel A,2,313.02,626.04,2019-11-14
13003,900,2,Hotel A,2,313.02,626.04,2020-02-06
13004,900,2,Hotel A,4,313.02,1252.08,2020-03-12
13005,900,2,Hotel A,2,313.02,626.04,2020-11-26
13006,900,2,Hotel A,3,313.02,939.06,2019-10-24
13007,900,2,Hotel A,1,313.02,313.02,2020-04-23
13008,900,2,Hotel A,2,313.02,626.04,2021-01-28
13009,900,2,Hotel A,1,313.02,313.02,2021-06-24


In [159]:
flighttype

,flightTypeCode,flightTypeName
0,100,firstClass
1,101,economic
2,102,premium


In [160]:
agency

,agencyCode,agencyName
0,400,FlyingDrops
1,401,CloudFy
2,402,Rainbow


In [161]:
travel.head()

,travelCode,userCode,fromLocationCode,toLocationCode,flightTypeCode,price,time,distance,agencyCode,date
0,0,0,1,2,100,1434.38,1.76,676.53,400,2019-09-26
1,0,0,2,1,100,1292.29,1.76,676.53,400,2019-09-30
2,1,0,3,2,100,1487.52,1.66,637.56,401,2019-10-03
3,1,0,2,3,100,1127.36,1.66,637.56,401,2019-10-04
4,2,0,4,5,100,1684.05,2.16,830.86,401,2019-10-10


saving csv


In [162]:
user.columns = user.columns.str.lower()


In [134]:
user

,userCode,name,age,gender,companyCode
0,0,Roy Braun,21,M,700
1,1,Joseph Holsten,37,M,700
2,2,Wilma Mcinnis,48,F,700
3,3,Paula Daniel,23,F,700
4,4,Patricia Carson,44,F,700
...,...,...,...,...,...
1335,1335,Albert Garroutte,23,M,704
1336,1336,Kim Shores,40,F,704
1337,1337,James Gimenez,28,M,704
1338,1338,Viola Agosta,52,F,704


In [163]:
user.to_csv(r"D:\study materials\aws\final all codes\csv_files\users.csv", index = False)

In [197]:
gender.columns = gender.columns.str.lower()

In [198]:
gender.to_csv(r"D:\study materials\aws\final all codes\csv_files\gender.csv", index = False)

In [166]:
location.columns = location.columns.str.lower()

In [167]:
location.to_csv(r"D:\study materials\aws\final all codes\csv_files\location.csv", index = False)

In [119]:
accommodation.columns = accommodation.columns.str.lower()

In [120]:
accommodation.to_csv(r"D:\study materials\aws\final all codes\csv_files\accommodation.csv", index = False)

In [170]:
flighttype.columns = flighttype.columns.str.lower()

In [171]:
flighttype.to_csv(r"D:\study materials\aws\final all codes\csv_files\flighttype.csv", index = False)

In [172]:
agency.columns = agency.columns.str.lower()

In [173]:
agency.to_csv(r"D:\study materials\aws\final all codes\csv_files\agency.csv", index = False)

In [121]:
travel.columns = travel.columns.str.lower()

In [122]:
travel.to_csv(r"D:\study materials\aws\final all codes\csv_files\travel.csv", index = False)

In [176]:
hotel.columns = hotel.columns.str.lower()

In [177]:
hotel.to_csv(r"D:\study materials\aws\final all codes\csv_files\hotel.csv", index = False)

In [84]:
bucket = 'sundardl-travel-data'

In [ ]:
''' StringIO() is a temporary place in memory (like a fake file).
We're using it to store the CSV content without saving it on disk.'''

csv_buffer = StringIO() 
csv_buffer

In [ ]:
''' This writes your pandas DataFrame (users) into the csv_buffer '''
users.to_csv(csv_buffer)

In [ ]:
s3 = boto3.resource(
    's3',
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
    region_name='ap-south-1'
)

s3.Object(bucket, 'output/users.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'FVS5A1VCSGJDZT69',
  'HostId': '/YxL0PEbOKjotG9mrNlYhjCGc7hTjm0+MC+2VoPYExKB9neEc+1NNK1Ay09XkekLIuFcDKLeB3sItSW+0ZcGMA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/YxL0PEbOKjotG9mrNlYhjCGc7hTjm0+MC+2VoPYExKB9neEc+1NNK1Ay09XkekLIuFcDKLeB3sItSW+0ZcGMA==',
   'x-amz-request-id': 'FVS5A1VCSGJDZT69',
   'date': 'Sun, 29 Jun 2025 05:07:09 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"df546c9429f19912d43f2dd07d519bb5"',
   'x-amz-checksum-crc32': 'c9nj5w==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"df546c9429f19912d43f2dd07d519bb5"',
 'ChecksumCRC32': 'c9nj5w==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

schema

In [123]:
user_schema = pd.io.sql.get_schema(user, 'users')
print(''.join(user_schema))

CREATE TABLE "users" (
"userCode" INTEGER,
  "name" TEXT,
  "age" INTEGER,
  "gender" TEXT,
  "companyCode" INTEGER
)


In [124]:
gender_schema = pd.io.sql.get_schema(gender, 'gender')
print(''.join(gender_schema))

CREATE TABLE "gender" (
"genderCode" TEXT,
  "genderName" TEXT
)


In [125]:
location_schema  = pd.io.sql.get_schema(location, 'location')
print(''.join(location_schema))

CREATE TABLE "location" (
"locationCode" INTEGER,
  "locationName" TEXT
)


In [126]:
accommodation_schema = pd.io.sql.get_schema(accommodation, 'accommodation')
print(''.join(accommodation_schema))

CREATE TABLE "accommodation" (
"accommodationid" INTEGER,
  "usercode" INTEGER,
  "hotelcode" INTEGER,
  "locationcode" INTEGER,
  "name" TEXT,
  "days" INTEGER,
  "price" REAL,
  "total" REAL,
  "date" TIMESTAMP
)


In [127]:
flighttype_schema = pd.io.sql.get_schema(flighttype, 'flighttype')
print(''.join(flighttype_schema))

CREATE TABLE "flighttype" (
"flightTypeCode" TEXT,
  "flightTypeName" TEXT
)


In [128]:
agency_schema = pd.io.sql.get_schema(agency, 'agency')
print(''.join(agency_schema))

CREATE TABLE "agency" (
"agencyCode" TEXT,
  "agencyName" TEXT
)


In [129]:
travel_schema = pd.io.sql.get_schema(travel, 'travel')
print(''.join(travel_schema))

CREATE TABLE "travel" (
"travelcode" INTEGER,
  "usercode" INTEGER,
  "fromlocationcode" INTEGER,
  "tolocationcode" INTEGER,
  "flighttypecode" TEXT,
  "price" REAL,
  "time" REAL,
  "distance" REAL,
  "agencycode" TEXT,
  "date" TIMESTAMP
)


In [130]:
hotel_schema = pd.io.sql.get_schema(hotel, 'hotel')
print(''.join(hotel_schema))

CREATE TABLE "hotel" (
"hotelCode" INTEGER,
  "hotelName" TEXT
)


In [ ]:
''' This script connects to AWS S3, searches for a specific file (or files starting with 'output/users.csv') 
 inside the 'sundardl-travel-data' bucket, and prints the file name (key) and size in bytes.'''

import boto3

s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket='sundardl-travel-data', Prefix='output/users.csv')

for obj in response.get('Contents', []):
    print(f"File: {obj['Key']}, Size: {obj['Size']} bytes")


File: output/users.csv, Size: 57314 bytes


In [99]:
import boto3

client = boto3.client('redshift-data', region_name='ap-south-1', aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'))

response = client.execute_statement(
    WorkgroupName='redshift-workgroup',
    Database='dev',
    SecretArn='arn:aws:secretsmanager:ap-south-1:569280684615:secret:RedshiftServerlessTravelDataSecret-creVHd',
    Sql="""
    CREATE TABLE "users" (
    "userCode" INTEGER,
    "name" TEXT,
    "age" INTEGER,
    "gender" TEXT,
    "companyCode" INTEGER
    )
    """
)

In [179]:
import boto3

s3 = boto3.client('s3')
s3.head_object(Bucket='sundardl-travel-data', Key='output/users.csv')


{'ResponseMetadata': {'RequestId': 'V89SVMTZ55Y858WF',
  'HostId': 'cu0is9ezOnOudAMl3EYMNEYmU+DDWgrtIo45RmXEQTohLjaqBW+JCdcT6jqzb/OgUjAgtZGr8apDs4Asz8lb1g==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cu0is9ezOnOudAMl3EYMNEYmU+DDWgrtIo45RmXEQTohLjaqBW+JCdcT6jqzb/OgUjAgtZGr8apDs4Asz8lb1g==',
   'x-amz-request-id': 'V89SVMTZ55Y858WF',
   'date': 'Sat, 28 Jun 2025 14:30:13 GMT',
   'last-modified': 'Sat, 28 Jun 2025 11:44:16 GMT',
   'etag': '"df546c9429f19912d43f2dd07d519bb5"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '57314',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2025, 6, 28, 11, 44, 16, tzinfo=tzutc()),
 'ContentLength': 57314,
 'ETag': '"df546c9429f19912d43f2dd07d519bb5"',
 'ContentType': 'binary/octet-stream',
 'ServerSideEncryption': 'AES256',
 'Metadata': {}}

In [101]:
import boto3

client = boto3.client('redshift-data', region_name='ap-south-1', aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'))

response = client.execute_statement(
    WorkgroupName='redshift-workgroup',
    Database='dev',
    SecretArn='arn:aws:secretsmanager:ap-south-1:569280684615:secret:RedshiftServerlessTravelDataSecret-creVHd',
    Sql="""
    COPY users
    FROM 's3://sundardl-travel-data/output/users.csv'
    IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
    REGION 'ap-south-1'
    IGNOREHEADER 1
    CSV
    ;
    """
)

print("✅ COPY command submitted. Query ID:", response)


✅ COPY command submitted. Query ID: {'CreatedAt': datetime.datetime(2025, 6, 29, 10, 59, 35, 493000, tzinfo=tzlocal()), 'Database': 'dev', 'Id': '2455c07a-9cc0-40e1-b107-7abf7e3fb733', 'SecretArn': 'arn:aws:secretsmanager:ap-south-1:569280684615:secret:RedshiftServerlessTravelDataSecret-creVHd', 'WorkgroupName': 'redshift-workgroup', 'ResponseMetadata': {'RequestId': '2455c07a-9cc0-40e1-b107-7abf7e3fb733', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2455c07a-9cc0-40e1-b107-7abf7e3fb733', 'content-type': 'application/x-amz-json-1.1', 'content-length': '238', 'date': 'Sun, 29 Jun 2025 05:29:35 GMT'}, 'RetryAttempts': 0}}


In [ ]:
# glue job 
import boto3
import time


client = boto3.client('redshift-data', region_name='ap-south-1',
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
)

SECRET_ARN = 'arn:aws:secretsmanager:ap-south-1:569280684615:secret:RedshiftServerlessTravelDataSecret-creVHd'
WORKGROUP = 'redshift-workgroup'
DATABASE = 'dev'



def run_redshift_sql(sql, label=""):
    print(f"Submitting: {label} ...")
    response = client.execute_statement(
        WorkgroupName=WORKGROUP,
        Database=DATABASE,
        SecretArn=SECRET_ARN,
        Sql=sql
    )
    query_id = response['Id']


    while True:
        status = client.describe_statement(Id=query_id)["Status"]
        if status in ["FINISHED", "FAILED", "ABORTED"]:
            print(f"label} completed with status: {status}")
            return status
        time.sleep(2)



create_table_users = """
CREATE TABLE IF NOT EXISTS users (
    usercode INTEGER,
    name TEXT,
    age INTEGER,
    gender TEXT,
    companycode INTEGER
);
"""

create_table_users = """
CREATE TABLE IF NOT EXISTS users (
    usercode INTEGER,
    name TEXT,
    age INTEGER,
    gender TEXT,
    companycode INTEGER
);
"""


copy_users = """
COPY users
FROM 's3://sundardl-travel-data/output/users.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""



run_redshift_sql(create_table_users, label="CREATE TABLE users")
run_redshift_sql(copy_users, label="COPY users from S3")



create_table_gender = """
CREATE TABLE IF NOT EXISTS "gender" (
"gendercode" TEXT,
"gendername" TEXT
)
"""

copy_gender = """
COPY gender
FROM 's3://sundardl-travel-data/output/gender.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_gender, label="CREATE TABLE gender")
run_redshift_sql(copy_gender, label="COPY gender from S3")





create_table_location = """
CREATE TABLE IF NOT EXISTS "location" (
"locationcode" INTEGER,
  "locationname" TEXT
)
"""

copy_location = """
COPY location
FROM 's3://sundardl-travel-data/output/location.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_location, label="CREATE TABLE location")
run_redshift_sql(copy_location, label="COPY location from S3")





create_table_accommodation = """
CREATE TABLE "accommodation" (
"accommodationid" INTEGER,
  "usercode" INTEGER,
  "hotelcode" INTEGER,
  "locationcode" INTEGER,
  "name" TEXT,
  "days" INTEGER,
  "price" REAL,
  "total" REAL,
  "date" TIMESTAMP
)
)
"""

copy_accommodation = """
COPY accommodation
FROM 's3://sundardl-travel-data/output/accommodation.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_accommodation, label="CREATE TABLE accommodation")
run_redshift_sql(copy_accommodation, label="COPY accommodation from S3")







create_table_flighttype = """
CREATE TABLE IF NOT EXISTS "flighttype" (
"flighttypecode" TEXT,
  "flighttypename" TEXT
)
"""

copy_flighttype = """
COPY flighttype
FROM 's3://sundardl-travel-data/output/flighttype.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_flighttype, label="CREATE TABLE flighttype")
run_redshift_sql(copy_flighttype, label="COPY flighttype from S3")







create_table_agency = """
CREATE TABLE IF NOT EXISTS "agency" (
"agencycode" TEXT,
  "agencyname" TEXT
)
"""

copy_agency = """
COPY agency
FROM 's3://sundardl-travel-data/output/agency.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_agency, label="CREATE TABLE agency")
run_redshift_sql(copy_agency, label="COPY agency from S3")





create_table_travel = """
CREATE TABLE "travel" (
"travelcode" INTEGER,
  "usercode" INTEGER,
  "fromlocationcode" INTEGER,
  "tolocationcode" INTEGER,
  "flighttypecode" TEXT,
  "price" REAL,
  "time" REAL,
  "distance" REAL,
  "agencycode" TEXT,
  "date" TIMESTAMP
)
"""

copy_travel = """
COPY travel
FROM 's3://sundardl-travel-data/output/travel.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_travel, label="CREATE TABLE travel")
run_redshift_sql(copy_travel, label="COPY travel from S3")






create_table_hotel = """
CREATE TABLE IF NOT EXISTS "hotel" (
"hotelcode" INTEGER,
  "hotelname" TEXT
)
"""

copy_hotel = """
COPY hotel
FROM 's3://sundardl-travel-data/output/hotel.csv'
IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
REGION 'ap-south-1'
IGNOREHEADER 1
CSV;
"""

run_redshift_sql(create_table_hotel, label="CREATE TABLE hotel")
run_redshift_sql(copy_hotel, label="COPY hotel from S3")


In [ ]:
subnet-04baba2b1490b09e4

In [ ]:
subnet-0c86705e88f1d427a

In [ ]:
import boto3
import time

client = boto3.client('redshift-data', region_name='ap-south-1',
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY')
)

SECRET_ARN = 'arn:aws:secretsmanager:ap-south-1:569280684615:secret:RedshiftServerlessTravelDataSecret-creVHd'
WORKGROUP = 'redshift-workgroup'
DATABASE = 'dev'

def run_sql(sql, label):
    print(f"{label} ...")
    response = client.execute_statement(
        WorkgroupName=WORKGROUP,
        Database=DATABASE,
        SecretArn=SECRET_ARN,
        Sql=sql
    )
    query_id = response['Id']
    while True:
        status = client.describe_statement(Id=query_id)['Status']
        if status in ['FINISHED', 'FAILED', 'ABORTED']:
            print(f"{label} => {status}")
            break
        time.sleep(2)

tables = {
    "users": """
        CREATE TABLE IF NOT EXISTS users (
            usercode INTEGER,
            name TEXT,
            age INTEGER,
            gender TEXT,
            companycode INTEGER
        );
    """,
    "gender": """
        CREATE TABLE IF NOT EXISTS gender (
            gendercode TEXT,
            gendername TEXT
        );
    """,
    "location": """
        CREATE TABLE IF NOT EXISTS location (
            locationcode INTEGER,
            locationname TEXT
        );
    """,
    "accommodation": """
        CREATE TABLE "accommodation" (
        "accommodationid" INTEGER,
        "usercode" INTEGER,
        "hotelcode" INTEGER,
        "locationcode" INTEGER,
        "name" TEXT,
        "days" INTEGER,
        "price" REAL,
        "total" REAL,
        "date" TIMESTAMP
        );
    """,
    "flighttype": """
        CREATE TABLE IF NOT EXISTS flighttype (
            flighttypecode TEXT,
            flighttypename TEXT
        );
    """,
    "agency": """
        CREATE TABLE IF NOT EXISTS agency (
            agencycode TEXT,
            agencyname TEXT
        );
    """,
    "travel": """
        CREATE TABLE "travel" (
        "travelcode" INTEGER,
        "usercode" INTEGER,
        "fromlocationcode" INTEGER,
        "tolocationcode" INTEGER,
        "flighttypecode" TEXT,
        "price" REAL,
        "time" REAL,
        "distance" REAL,
        "agencycode" TEXT,
        "date" TIMESTAMP
        );
    """,
    "hotel": """
        CREATE TABLE IF NOT EXISTS hotel (
            hotelcode INTEGER,
            hotelname TEXT
        );
    """
}

for table_name, create_sql in tables.items():
    run_sql(create_sql, f"CREATE TABLE {table_name}")
    copy_sql = f"""
        COPY {table_name}
        FROM 's3://sundardl-travel-data/output/{table_name}.csv'
        IAM_ROLE 'arn:aws:iam::569280684615:role/s3-glue-role'
        REGION 'ap-south-1'
        IGNOREHEADER 1
        CSV;
    """
    run_sql(copy_sql, f"COPY {table_name}")


postgres create and insert data

In [131]:
import os
import psycopg2
import pandas as pd

# Path where all your CSV files are stored
csv_folder = r"D:\study materials\aws\final all codes\csv_files"

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="travel_dataset",
    user="postgres",
    password="postgres"
)
cursor = conn.cursor()

create_statements = {
    "users": """
        CREATE TABLE IF NOT EXISTS users (
            usercode INTEGER,
            name TEXT,
            age INTEGER,
            gender TEXT,
            companycode INTEGER
        );
    """,
    "gender": """
        CREATE TABLE IF NOT EXISTS gender (
            gendercode TEXT,
            gendername TEXT
        );
    """,
    "location": """
        CREATE TABLE IF NOT EXISTS location (
            locationcode INTEGER,
            locationname TEXT
        );
    """,
    "accommodation": """
        CREATE TABLE "accommodation" (
        "accommodationid" INTEGER,
        "usercode" INTEGER,
        "hotelcode" INTEGER,
        "locationcode" INTEGER,
        "name" TEXT,
        "days" INTEGER,
        "price" REAL,
        "total" REAL,
        "date" TIMESTAMP
        );
    """,
    "flighttype": """
        CREATE TABLE IF NOT EXISTS flighttype (
            flighttypecode TEXT,
            flighttypename TEXT
        );
    """,
    "agency": """
        CREATE TABLE IF NOT EXISTS agency (
            agencycode TEXT,
            agencyname TEXT
        );
    """,
    "travel": """
        CREATE TABLE "travel" (
        "travelcode" INTEGER,
        "usercode" INTEGER,
        "fromlocationcode" INTEGER,
        "tolocationcode" INTEGER,
        "flighttypecode" TEXT,
        "price" REAL,
        "time" REAL,
        "distance" REAL,
        "agencycode" TEXT,
        "date" TIMESTAMP
        );
    """,
    "hotel": """
        CREATE TABLE IF NOT EXISTS hotel (
            hotelcode INTEGER,
            hotelname TEXT
        );
    """
}


for table, create_sql in create_statements.items():
    cursor.execute(create_sql)
    conn.commit()
    print(f"Created table: {table}")

# Step 3: Insert CSV data into tables
for table in create_statements.keys():
    csv_path = os.path.join(csv_folder, f"{table}.csv")
    print(f"Inserting into table: {table} from {csv_path}")
    df = pd.read_csv(csv_path)

    # Dynamically build insert query
    columns = ','.join(df.columns)
    placeholders = ','.join(['%s'] * len(df.columns))
    insert_sql = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"

    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_sql, row)

    conn.commit()
    print(f"Inserted {len(df)} rows into {table}.")

cursor.close()
conn.close()
print("✅ All tables created and data inserted.")


Created table: users
Created table: gender
Created table: location
Created table: accommodation
Created table: flighttype
Created table: agency
Created table: travel
Created table: hotel
Inserting into table: users from D:\study materials\aws\final all codes\csv_files\users.csv
Inserted 1340 rows into users.
Inserting into table: gender from D:\study materials\aws\final all codes\csv_files\gender.csv
Inserted 3 rows into gender.
Inserting into table: location from D:\study materials\aws\final all codes\csv_files\location.csv
Inserted 9 rows into location.
Inserting into table: accommodation from D:\study materials\aws\final all codes\csv_files\accommodation.csv
Inserted 40552 rows into accommodation.
Inserting into table: flighttype from D:\study materials\aws\final all codes\csv_files\flighttype.csv
Inserted 3 rows into flighttype.
Inserting into table: agency from D:\study materials\aws\final all codes\csv_files\agency.csv
Inserted 3 rows into agency.
Inserting into table: travel fro